# Secondary Outcomes
**S1. Cost per 1,000 patients for top 3 pre-specified “low-priority” treatments combined.**

**S2. Total items prescribed per 1000 registered patients for Co-proxamol.**
 
**S3. Total items prescribed per 1000 registered patients for Dosulepin.**

In [1]:
import os
import requests
import pandas as pd
import numpy as np

from analysis import compute_regression

GBQ_PROJECT_ID = '620265099307'

# Set dates of baseline and follow-up periods
baseline_start = '2018-04-01'       # baseline start
mid_start = '2018-10-01'            # month after end of baseline period
followup_start = '2019-04-01'       # follow-up start
post_followup_start = '2019-10-01'  # month after end of follow-up period


In [2]:
rawdata = pd.read_csv(os.path.join('..','data','all_measure_data.csv'))
rawdata["month"] = pd.to_datetime(rawdata.month)
rawdata.head()

,Unnamed: 0,month,pct_id,items,cost,denominator,measure
0,0,2018-10-01 00:00:00+00:00,00C,4,392.31788,108.634,lpcoprox
1,1,2018-11-01 00:00:00+00:00,00C,2,127.21812,108.594,lpcoprox
2,2,2018-12-01 00:00:00+00:00,00C,6,551.13563,108.595,lpcoprox
3,3,2018-10-01 00:00:00+00:00,00D,1,39.54454,292.679,lpcoprox
4,4,2018-11-01 00:00:00+00:00,00D,3,345.62081,292.747,lpcoprox


## S1. Cost per 1,000 patients for top 3 pre-specified “low-priority” treatments combined. 

In [3]:
data = rawdata.copy()

### select data only for the baseline and follow-up periods
import datetime

conditions = [
    (data['month'] >= post_followup_start),
    (data['month'] >= followup_start),
    (data['month'] >= mid_start),
    (data['month'] >= baseline_start),
    (data['month'] < baseline_start)]

choices = ['after', 'follow-up', 'mid', 'baseline','before']
data['period'] = np.select(conditions, choices, default='0')

# take columns of interest from df
df2 = data[["measure","pct_id","period", "month", "cost","items","denominator"]]
df2 = df2.loc[(df2['period']== "baseline") | (df2['period']== "follow-up")].set_index(["pct_id","period", "month"])
df2.head()

measure       cost  items  \
pct_id period   month                                                   
00C    baseline 2018-04-01 00:00:00+00:00  lpcoprox  587.41623      8   
                2018-05-01 00:00:00+00:00  lpcoprox   55.69691      1   
                2018-06-01 00:00:00+00:00  lpcoprox  255.93032      3   
00D    baseline 2018-04-01 00:00:00+00:00  lpcoprox  552.77494      4   
                2018-05-01 00:00:00+00:00  lpcoprox    0.00000      0   

                                           denominator  
pct_id period   month                                   
00C    baseline 2018-04-01 00:00:00+00:00      108.273  
                2018-05-01 00:00:00+00:00      108.299  
                2018-06-01 00:00:00+00:00      108.356  
00D    baseline 2018-04-01 00:00:00+00:00      292.023  
                2018-05-01 00:00:00+00:00      292.107

In [4]:
### sum numerator and average population denominators for each CCG for each period
agg_6m = df2.groupby(["measure","pct_id","period"]).agg({"cost":sum,"items":sum,"denominator":"mean"})
agg_6m.head()

cost  items  denominator
measure  pct_id period                                    
lpcoprox 00C    baseline    1682.94526     23   108.418333
                follow-up   2128.24541     30   108.911500
         00D    baseline     561.78717      5   292.331833
                follow-up      0.00000      0   293.096833
         00J    baseline   18105.34852    111   259.248000

In [5]:
### import **allocated** CCGs
ccgs = pd.read_csv(os.path.join('..','data','randomisation_group.csv'))
# import joint team information
team = pd.read_csv(os.path.join('..','data','joint_teams.csv'))

ccgs = ccgs.merge(team,on="joint_team", how="left")
#fill black ccg_ids from joint_id column
ccgs["pct_id"] = ccgs["ccg_id"].combine_first(ccgs["joint_id"])
ccgs = ccgs[["joint_id","allocation","pct_id"]]

df2b = agg_6m.reset_index()
df2b = ccgs.merge(df2b, on="pct_id",how="left")
df2b.head()


,joint_id,allocation,pct_id,measure,period,cost,items,denominator
0,01X,con,01X,lpcoprox,baseline,7845.14647,39,197.704333
1,01X,con,01X,lpcoprox,follow-up,1993.56240,14,198.111167
2,01X,con,01X,lpdosulepin,baseline,4167.29525,2051,197.704333
3,01X,con,01X,lpdosulepin,follow-up,20835.21100,1686,198.111167
4,01X,con,01X,lpdoxazosin,baseline,4673.51929,492,197.704333


In [6]:
# group up to Joint team groups 
# note: SUM both numerators and population denominator across geographies
df2c = df2b.groupby(["joint_id","allocation","measure","period"]).sum()
df2c = df2c.unstack().reset_index()
df2c.columns = df2c.columns.map('_'.join)

### calculate aggregated measure values (cost only)
df2c["baseline_calc_value"] = df2c.cost_baseline / df2c.denominator_baseline
df2c["follow_up_calc_value"] = df2c["cost_follow-up"] / df2c["denominator_follow-up"]

df2c.head()


,joint_id_,allocation_,measure_,cost_baseline,cost_follow-up,items_baseline,items_follow-up,denominator_baseline,denominator_follow-up,baseline_calc_value,follow_up_calc_value
0,00J,I,lpcoprox,18105.34852,10348.92981,111,47,259.248,261.553833,69.837949,39.567112
1,00J,I,lpdosulepin,1912.89594,6333.01366,1266,795,259.248,261.553833,7.378633,24.213041
2,00J,I,lpdoxazosin,6478.96813,6131.91956,703,710,259.248,261.553833,24.991391,23.444197
3,00J,I,lpfentanylir,23649.01996,29199.73282,110,83,259.248,261.553833,91.221610,111.639476
4,00J,I,lpglucosamine,6726.00966,2875.00410,299,213,259.248,261.553833,25.944307,10.992017


In [7]:
# find top 3 measures per CCG by cost
df3 = df2c.sort_values(by=["joint_id_","baseline_calc_value"], ascending=False)
df3["measure_rank"] = df3.groupby("joint_id_")["baseline_calc_value"].rank(ascending=False)
df4 = df3.loc[df3.measure_rank <=3]
df4.head()

,joint_id_,allocation_,measure_,cost_baseline,cost_follow-up,items_baseline,items_follow-up,denominator_baseline,denominator_follow-up,baseline_calc_value,follow_up_calc_value,measure_rank
705,99K,con,lpfentanylir,52079.50137,24239.10571,119,70,170.713667,171.033667,305.069315,141.721254,1.0
710,99K,con,lpliothyronine,50050.90100,33104.70501,155,110,170.713667,171.033667,293.186257,193.556658,2.0
719,99K,con,lptrimipramine,48736.87774,52512.58144,164,146,170.713667,171.033667,285.489022,307.030671,3.0
692,99J,con,lpliothyronine,209433.01209,136225.25598,794,638,493.998000,500.103500,423.955182,272.394126,1.0
701,99J,con,lptrimipramine,168361.17675,160905.19318,445,375,493.998000,500.103500,340.813478,321.743785,2.0


In [17]:
# check whether any CCGs' top 3 include herbal medicine which was not available as a measure at the time of the intervention
df4.loc[df4["measure_"]=="lpherbal"]

,joint_id_,allocation_,measure_,cost_baseline,cost_follow-up,items_baseline,items_follow-up,denominator_baseline,denominator_follow-up,baseline_calc_value,follow_up_calc_value,measure_rank


In [8]:
df5 = df4.copy()
df5 = df5.groupby(["joint_id_","allocation_"]).agg({"cost_baseline":"sum","cost_follow-up":"sum","denominator_baseline":"mean","denominator_follow-up":"mean"})

### calculate aggregated measure values for combined cost for the top 3 measures
df5["baseline_calc_value"] = df5.cost_baseline / df5.denominator_baseline
df5["follow_up_calc_value"] = df5["cost_follow-up"] / df5["denominator_follow-up"]
df5.head() 


,,cost_baseline,cost_follow-up,denominator_baseline,denominator_follow-up,baseline_calc_value,follow_up_calc_value
joint_id_,allocation_,,,,,,
00J,I,179617.41370,161140.38067,259.248000,261.553833,692.840113,616.088775
00Y,con,140982.13270,65189.09093,255.777833,258.976833,551.189800,251.717847
01F,con,163150.55569,105996.32564,131.572667,132.790333,1240.003413,798.223206
01J,con,203730.64187,158096.42026,165.419167,166.806000,1231.602395,947.786172
01V,con,300578.38709,247990.96216,280.439333,281.692833,1071.812515,880.359501


In [9]:
# secondary outcome: Cost per 1,000 patients for top 3 pre-specified “low-priority” treatments combined.

import statsmodels.formula.api as smf
data = df5.copy().reset_index()
# create a new Series called "intervention" to convert intervention/control to numerical values
data['intervention'] = data.allocation_.map({'con':0, 'I':1})

out = data.groupby("intervention").agg({"joint_id_":"nunique",
                                 "baseline_calc_value":{"mean","std"},
                                 "follow_up_calc_value":{"mean","std"}})

out["change"] = out[("follow_up_calc_value","mean")] - out[("baseline_calc_value","mean")]

display(out)

formula = ('data["follow_up_calc_value"] ~ data["baseline_calc_value"] +intervention')
compute_regression(data, formula=formula)


joint_id_ baseline_calc_value             follow_up_calc_value  \
               nunique                 std        mean                  std   
intervention                                                                  
0                   20          225.697333  866.845267           206.789446   
1                   20          159.695379  756.709713           157.760994   

                              change  
                    mean              
intervention                          
0             664.082485 -202.762782  
1             592.244016 -164.465698

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                 
========================================================================================
Dep. Variable:     data["follow_up_calc_value"]   R-squared:                       0.728
Model:                                      OLS   Adj. R-squared:                  0.713
Method:                           Least Squares   F-statistic:                     49.49
Date:                          Thu, 19 Nov 2020   Prob (F-statistic):           3.49e-11
Time:                                  15:42:52   Log-Likelihood:                -239.07
No. Observations:                            40   AIC:                             484.1
Df Residuals:                                37   BIC:                             489.2
Df Model:                                     2                                         
Covariance Type:                      nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                     -26.4092     74.691     -0.354      0.726    -177.747     124.929
data["baseline_calc_value"]     0.7966      0.082      9.681      0.000       0.630       0.963
intervention                   15.8908     32.640      0.487      0.629     -50.245      82.026
==============================================================================
Omnibus:                        3.158   Durbin-Watson:                   2.059
Prob(Omnibus):                  0.206   Jarque-Bera (JB):                2.927
Skew:                          -0.636   Prob(JB):                        0.231
Kurtosis:                       2.627   Cond. No.                     4.08e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.08e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## S2: Total items prescribed per 1000 registered patients for Co-proxamol. 

In [10]:
# filter data for coproxamol measure:
df6 = df2c.copy()
df6 = df6.loc[df6.measure_=="lpcoprox"]

### calculate aggregated measure values (items per 1000 patients)
df6["baseline_calc_value"] = df6.items_baseline / df6.denominator_baseline
df6["follow_up_calc_value"] = df6["items_follow-up"] / df6["denominator_follow-up"]
df6.head()

,joint_id_,allocation_,measure_,cost_baseline,cost_follow-up,items_baseline,items_follow-up,denominator_baseline,denominator_follow-up,baseline_calc_value,follow_up_calc_value
0,00J,I,lpcoprox,18105.34852,10348.92981,111,47,259.248000,261.553833,0.428161,0.179695
18,00Y,con,lpcoprox,7201.93514,6073.37376,46,37,255.777833,258.976833,0.179844,0.142870
36,01F,con,lpcoprox,2144.71350,2289.76199,12,9,131.572667,132.790333,0.091204,0.067776
54,01J,con,lpcoprox,2932.11798,1812.50430,24,18,165.419167,166.806000,0.145086,0.107910
72,01V,con,lpcoprox,4636.03187,2604.61799,42,19,280.439333,281.692833,0.149765,0.067449


In [11]:
## Secondary outcome: Total items prescribed per 1000 registered patients for Co-proxamol.
import statsmodels.formula.api as smf
data = df6.copy().reset_index()
# create a new Series called "intervention" to convert intervention/control to numerical values
data['intervention'] = data.allocation_.map({'con':0, 'I':1})


out = data.groupby("intervention").agg({"joint_id_":"nunique",
                                 "baseline_calc_value":{"mean","std"},
                                 "follow_up_calc_value":{"mean","std"}})

out["change"] = out[("follow_up_calc_value","mean")] - out[("baseline_calc_value","mean")]

display(out)

formula = ('data["follow_up_calc_value"] ~ data["baseline_calc_value"] +intervention')
compute_regression(data, formula=formula)

joint_id_ baseline_calc_value           follow_up_calc_value  \
               nunique                 std      mean                  std   
intervention                                                                
0                   20            0.159883  0.295355             0.122855   
1                   20            0.231633  0.262386             0.203357   

                          change  
                  mean            
intervention                      
0             0.198990 -0.096364  
1             0.176306 -0.086081

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                 
========================================================================================
Dep. Variable:     data["follow_up_calc_value"]   R-squared:                       0.844
Model:                                      OLS   Adj. R-squared:                  0.836
Method:                           Least Squares   F-statistic:                     100.4
Date:                          Thu, 19 Nov 2020   Prob (F-statistic):           1.12e-15
Time:                                  15:42:52   Log-Likelihood:                 52.743
No. Observations:                            40   AIC:                            -99.49
Df Residuals:                                37   BIC:                            -94.42
Df Model:                                     2                                         
Covariance Type:                      nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                      -0.0300      0.022     -1.358      0.183      -0.075       0.015
data["baseline_calc_value"]     0.7754      0.055     14.134      0.000       0.664       0.887
intervention                    0.0029      0.021      0.135      0.894      -0.040       0.046
==============================================================================
Omnibus:                        0.116   Durbin-Watson:                   1.812
Prob(Omnibus):                  0.944   Jarque-Bera (JB):                0.157
Skew:                          -0.110   Prob(JB):                        0.924
Kurtosis:                       2.786   Cond. No.                         6.35
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## S3: Total items prescribed per 1000 registered patients for Dosulepin. 

In [12]:
# filter data for dosulepin measure:
df7 = df2c.copy()
df7 = df7.loc[df7.measure_=="lpdosulepin"]

### calculate aggregated measure values (items per 1000 patients)
df7["baseline_calc_value"] = df7.items_baseline / df7.denominator_baseline
df7["follow_up_calc_value"] = df7["items_follow-up"] / df7["denominator_follow-up"]
df7.head()

,joint_id_,allocation_,measure_,cost_baseline,cost_follow-up,items_baseline,items_follow-up,denominator_baseline,denominator_follow-up,baseline_calc_value,follow_up_calc_value
1,00J,I,lpdosulepin,1912.89594,6333.01366,1266,795,259.248000,261.553833,4.883355,3.039527
19,00Y,con,lpdosulepin,2537.48637,11897.58863,1476,1076,255.777833,258.976833,5.770633,4.154812
37,01F,con,lpdosulepin,2231.53336,10893.21717,1244,930,131.572667,132.790333,9.454851,7.003522
55,01J,con,lpdosulepin,3464.89964,20279.28536,2062,1774,165.419167,166.806000,12.465303,10.635109
73,01V,con,lpdosulepin,3022.75316,17020.62873,1535,1453,280.439333,281.692833,5.473555,5.158101


In [13]:
## Secondary outcome: Total items prescribed per 1000 registered patients for Dosulepin.
import statsmodels.formula.api as smf
data = df7.copy().reset_index()
# create a new Series called "intervention" to convert intervention/control to numerical values
data['intervention'] = data.allocation_.map({'con':0, 'I':1})

out = data.groupby("intervention").agg({"joint_id_":"nunique",
                                 "baseline_calc_value":{"mean","std"},
                                 "follow_up_calc_value":{"mean","std"}})

out["change"] = out[("follow_up_calc_value","mean")] - out[("baseline_calc_value","mean")]

display(out)

formula = ('data["follow_up_calc_value"] ~ data["baseline_calc_value"] +intervention')
compute_regression(data, formula=formula)

joint_id_ baseline_calc_value           follow_up_calc_value  \
               nunique                 std      mean                  std   
intervention                                                                
0                   20            2.774081  7.113155             2.427627   
1                   20            3.386770  6.465136             2.997525   

                          change  
                  mean            
intervention                      
0             6.058305 -1.054850  
1             5.475997 -0.989139

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                 
========================================================================================
Dep. Variable:     data["follow_up_calc_value"]   R-squared:                       0.958
Model:                                      OLS   Adj. R-squared:                  0.956
Method:                           Least Squares   F-statistic:                     424.1
Date:                          Thu, 19 Nov 2020   Prob (F-statistic):           3.09e-26
Time:                                  15:42:52   Log-Likelihood:                -32.605
No. Observations:                            40   AIC:                             71.21
Df Residuals:                                37   BIC:                             76.28
Df Model:                                     2                                         
Covariance Type:                      nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                      -0.0750      0.247     -0.304      0.763      -0.576       0.426
data["baseline_calc_value"]     0.8622      0.030     28.945      0.000       0.802       0.923
intervention                   -0.0236      0.181     -0.130      0.897      -0.390       0.343
==============================================================================
Omnibus:                        8.691   Durbin-Watson:                   1.902
Prob(Omnibus):                  0.013   Jarque-Bera (JB):                7.562
Skew:                          -0.923   Prob(JB):                       0.0228
Kurtosis:                       4.063   Cond. No.                         22.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""